In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import datetime
from scipy import stats
import re
import nltk
import math
import pickle

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

%matplotlib inline

## Data Preparation: Feature Engineering and Scaling


### Objective:

### - Prepare data for training the model
### - Transform all the categorical and numerical variables by imputing missing values, engineering outliers, and using one-hot encoding categorical variables.

### Load the data as a csv file

In [2]:
# data downloaded from kaggle https://www.kaggle.com/jsphyg/weather-dataset-rattle-package
df_weather = pd.read_csv('weatherAUS.csv')

In [3]:
df_weather.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [4]:
df_weather.shape

(145460, 23)

In [5]:
df_weather.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,143975.000000,144199.000000,142199.000000,82670.000000,75625.000000,135197.000000,143693.000000,142398.000000,142806.000000,140953.000000,130395.00000,130432.000000,89572.000000,86102.000000,143693.000000,141851.00000
mean,12.194034,23.221348,2.360918,5.468232,7.611178,40.035230,14.043426,18.662657,68.880831,51.539116,1017.64994,1015.255889,4.447461,4.509930,16.990631,21.68339
std,6.398495,7.119049,8.478060,4.193704,3.785483,13.607062,8.915375,8.809800,19.029164,20.795902,7.10653,7.037414,2.887159,2.720357,6.488753,6.93665
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.50000,977.100000,0.000000,0.000000,-7.200000,-5.40000
25%,7.600000,17.900000,0.000000,2.600000,4.800000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.90000,1010.400000,1.000000,2.000000,12.300000,16.60000
50%,12.000000,22.600000,0.000000,4.800000,8.400000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.60000,1015.200000,5.000000,5.000000,16.700000,21.10000
75%,16.900000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.40000,1020.000000,7.000000,7.000000,21.600000,26.40000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.00000,1039.600000,9.000000,9.000000,40.200000,46.70000


In [6]:
df_weather.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

In [7]:
df_weather.dtypes

Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow      object
dtype: object

### Drop the 'RainToday' column from the dataframe: Most likely the project will involve predicting if it rains tomorrow rather than today

In [8]:
df_weather.drop(['RainToday'], axis=1,inplace=True)

### 1. find out whether the target variable (“Rain_Tomorrow”) is categorical or numerical.

In [9]:
df_weather.dtypes

Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainTomorrow      object
dtype: object

In [10]:
# convert 'RainTomorrow' from object to category data type
df_weather['RainTomorrow'] = df_weather['RainTomorrow'].astype('category')

In [11]:
df_weather.dtypes

Date               object
Location           object
MinTemp           float64
MaxTemp           float64
Rainfall          float64
Evaporation       float64
Sunshine          float64
WindGustDir        object
WindGustSpeed     float64
WindDir9am         object
WindDir3pm         object
WindSpeed9am      float64
WindSpeed3pm      float64
Humidity9am       float64
Humidity3pm       float64
Pressure9am       float64
Pressure3pm       float64
Cloud9am          float64
Cloud3pm          float64
Temp9am           float64
Temp3pm           float64
RainTomorrow     category
dtype: object

In [12]:
df_weather.columns[df_weather.isna().any()].tolist()

['MinTemp',
 'MaxTemp',
 'Rainfall',
 'Evaporation',
 'Sunshine',
 'WindGustDir',
 'WindGustSpeed',
 'WindDir9am',
 'WindDir3pm',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am',
 'Pressure3pm',
 'Cloud9am',
 'Cloud3pm',
 'Temp9am',
 'Temp3pm',
 'RainTomorrow']

In [13]:
# count the Nan values for each column in dataframe
df_weather.isna().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainTomorrow      3267
dtype: int64

### There are a lot on Nan values in almost all the columns: How to deal with this issue?

In [14]:
# determne the unique values in target variable,'RainTomorrow'
print(df_weather['RainTomorrow'].unique().tolist())

['No', 'Yes', nan]


In [15]:
# Get the frequency distribution of values
counts = df_weather['RainTomorrow'].value_counts().to_dict()
print(counts)

{'No': 110316, 'Yes': 31877}


In [16]:
# Evaluate the percentage of the frequency distribution for each label.
relative_frequencies = df_weather['RainTomorrow'].value_counts(normalize=True)
print(relative_frequencies)

No     0.775819
Yes    0.224181
Name: RainTomorrow, dtype: float64


### Convert  yes/no labels to integers 1/0 to make calculations easier.

In [17]:
# Before converting to integers
df_weather.RainTomorrow.value_counts()

No     110316
Yes     31877
Name: RainTomorrow, dtype: int64

In [18]:
# replace target variable column nan values with mode BEFORE  converting to integers
df_weather['RainTomorrow'] = df_weather.RainTomorrow.fillna(df_weather['RainTomorrow'].mode()[0])

In [19]:
# after filling nan with mode value: No before converting target variable values into integers
df_weather.RainTomorrow.value_counts()

No     113583
Yes     31877
Name: RainTomorrow, dtype: int64

In [20]:
#df_weather.RainTomorrow.replace(('Yes', 'No'), (1, 0), inplace=True) 
df_weather['RainTomorrow'] = df_weather['RainTomorrow'].map({'Yes': 1, 'No': 0})

In [21]:
# After converting to integers
df_weather.RainTomorrow.value_counts()

0    113583
1     31877
Name: RainTomorrow, dtype: int64

### Create Month, Day, Year columns for dataframe

In [22]:
# Covert 'Date' column to a datetime object
df_weather['Date'] = df_weather['Date'].astype('datetime64')

/Users/scottlew/anaconda2/envs/py3/lib/python3.6/site-packages/pandas/core/internals.py:3462: FutureWarning: Passing in 'datetime64' dtype with no frequency is deprecated and will raise in a future version. Please pass in 'datetime64[ns]' instead.
  return self.apply('astype', dtype=dtype, **kwargs)


In [23]:
df_weather.dtypes

Date             datetime64[ns]
Location                 object
MinTemp                 float64
MaxTemp                 float64
Rainfall                float64
Evaporation             float64
Sunshine                float64
WindGustDir              object
WindGustSpeed           float64
WindDir9am               object
WindDir3pm               object
WindSpeed9am            float64
WindSpeed3pm            float64
Humidity9am             float64
Humidity3pm             float64
Pressure9am             float64
Pressure3pm             float64
Cloud9am                float64
Cloud3pm                float64
Temp9am                 float64
Temp3pm                 float64
RainTomorrow              int64
dtype: object

In [24]:
df_weather['month'] = df_weather.Date.dt.month

In [25]:
df_weather['day'] = df_weather.Date.dt.day

In [26]:
df_weather['year'] = df_weather.Date.dt.year

In [27]:
# sanity check on new day, month, year columns in dataframe
df_weather[['Date','year','month','day']].tail(10)

,Date,year,month,day
145450,2017-06-16,2017,6,16
145451,2017-06-17,2017,6,17
145452,2017-06-18,2017,6,18
145453,2017-06-19,2017,6,19
145454,2017-06-20,2017,6,20
145455,2017-06-21,2017,6,21
145456,2017-06-22,2017,6,22
145457,2017-06-23,2017,6,23
145458,2017-06-24,2017,6,24
145459,2017-06-25,2017,6,25


### Drop original 'Date' column from dataframe

In [28]:
df_weather.drop(['Date'], axis=1,inplace=True)

In [29]:
df_weather.dtypes

Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainTomorrow       int64
month              int64
day                int64
year               int64
dtype: object

In [30]:
df_weather.dtypes

Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainTomorrow       int64
month              int64
day                int64
year               int64
dtype: object

### 2. Impute the missing values in numerical and categorical variables

In [31]:
# select non-numerical columns with select_dtypes()
#df_cat = df_num =  df_weather.select_dtypes(exclude = 'number')
cols_cat = list(df_weather.select_dtypes(exclude = 'number').columns)

#print(cols_cat)
for item in cols_cat:
    print(item)

print(type(cols_cat))

Location
WindGustDir
WindDir9am
WindDir3pm
<class 'list'>


In [32]:
num_cols =  list(df_weather.select_dtypes(include='float64').columns)
print(num_cols)
print(type(num_cols))
print()
for item in num_cols:
    print(item)
    
print()
print(len(num_cols))

['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm']
<class 'list'>

MinTemp
MaxTemp
Rainfall
Evaporation
Sunshine
WindGustSpeed
WindSpeed9am
WindSpeed3pm
Humidity9am
Humidity3pm
Pressure9am
Pressure3pm
Cloud9am
Cloud3pm
Temp9am
Temp3pm

16


In [33]:
# drop RainTomorrow column from numerical column list

print(num_cols)
print()
print(len(num_cols))

['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm']

16


In [34]:
# count the Nan values for each column in dataframe
# before imputing with mean value
df_weather.isna().sum()

Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainTomorrow         0
month                0
day                  0
year                 0
dtype: int64

In [35]:
# use loop for fillna with mean of values in the same column for all numerical columns
for item in num_cols:
    df_weather[item].fillna(value=df_weather[item].mean(), inplace=True)

In [36]:
# count the Nan values for each column in dataframe
# after imputing with mean value
df_weather.isna().sum()

Location             0
MinTemp              0
MaxTemp              0
Rainfall             0
Evaporation          0
Sunshine             0
WindGustDir      10326
WindGustSpeed        0
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am         0
WindSpeed3pm         0
Humidity9am          0
Humidity3pm          0
Pressure9am          0
Pressure3pm          0
Cloud9am             0
Cloud3pm             0
Temp9am              0
Temp3pm              0
RainTomorrow         0
month                0
day                  0
year                 0
dtype: int64

### good only numerical columns were imputed with mean as desired, categorical columns: WindGustDir, WindDir9am & WindDir3pm are unchanged

### Convert object columns into categorical columns

In [37]:
# before
df_weather.dtypes

Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainTomorrow       int64
month              int64
day                int64
year               int64
dtype: object

In [38]:
# use loop to convert object columns to categorical data type
for item in cols_cat :
    
    df_weather[item] = df_weather[item].astype('category')

In [39]:
# after, verify conversion to categorical
df_weather.dtypes

Location         category
MinTemp           float64
MaxTemp           float64
Rainfall          float64
Evaporation       float64
Sunshine          float64
WindGustDir      category
WindGustSpeed     float64
WindDir9am       category
WindDir3pm       category
WindSpeed9am      float64
WindSpeed3pm      float64
Humidity9am       float64
Humidity3pm       float64
Pressure9am       float64
Pressure3pm       float64
Cloud9am          float64
Cloud3pm          float64
Temp9am           float64
Temp3pm           float64
RainTomorrow        int64
month               int64
day                 int64
year                int64
dtype: object

### 2. Impute the missing values in categorical variables:

In [40]:
missing_cat = ['WindGustDir','WindDir9am','WindDir3pm']

for item in missing_cat:
    print(df_weather[item].dtype)

category
category
category


In [41]:
# replace missing categorical values in data with mode

In [42]:
df_weather[missing_cat].mode()

,WindGustDir,WindDir9am,WindDir3pm
0,W,N,SE


In [43]:
for item in missing_cat:
    
    print(df_weather[item].mode()[0])
    print()


W

N

SE



In [44]:
#check nan values in data

df_weather.isna().sum()

Location             0
MinTemp              0
MaxTemp              0
Rainfall             0
Evaporation          0
Sunshine             0
WindGustDir      10326
WindGustSpeed        0
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am         0
WindSpeed3pm         0
Humidity9am          0
Humidity3pm          0
Pressure9am          0
Pressure3pm          0
Cloud9am             0
Cloud3pm             0
Temp9am              0
Temp3pm              0
RainTomorrow         0
month                0
day                  0
year                 0
dtype: int64

In [45]:
for item in missing_cat:
    df_weather[item].fillna(df_weather[item].mode()[0],inplace = True)

In [46]:
#check if categorical columns have nans replaced with mode

df_weather.isna().sum()

Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainTomorrow     0
month            0
day              0
year             0
dtype: int64

### Calculate the IQR, or Interquartile Range, for the shortlisted columns:

In [47]:
# from https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba
Q1 = df_weather.quantile(0.25)
Q3 = df_weather.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

MinTemp           9.100000
MaxTemp          10.200000
Rainfall          1.000000
Evaporation       1.468232
Sunshine          1.088822
WindGustSpeed    15.000000
WindSpeed9am     12.000000
WindSpeed3pm     11.000000
Humidity9am      26.000000
Humidity3pm      28.000000
Pressure9am       8.300000
Pressure3pm       8.300000
Cloud9am          3.000000
Cloud3pm          2.000000
Temp9am           9.200000
Temp3pm           9.500000
RainTomorrow      0.000000
month             6.000000
day              15.000000
year              4.000000
dtype: float64


In [48]:
df_weather.dtypes

Location         category
MinTemp           float64
MaxTemp           float64
Rainfall          float64
Evaporation       float64
Sunshine          float64
WindGustDir      category
WindGustSpeed     float64
WindDir9am       category
WindDir3pm       category
WindSpeed9am      float64
WindSpeed3pm      float64
Humidity9am       float64
Humidity3pm       float64
Pressure9am       float64
Pressure3pm       float64
Cloud9am          float64
Cloud3pm          float64
Temp9am           float64
Temp3pm           float64
RainTomorrow        int64
month               int64
day                 int64
year                int64
dtype: object

In [49]:
for col in num_cols:
    col_lower = col +'_lower'
    col_upper = col +'_upper'
    col_iqr = col + '_IQR'
    Q1_col = df_weather[col].quantile(0.25)
    Q3_col = df_weather[col].quantile(0.75)
    IQR_col = Q3_col - Q1_col
    df_weather[col_iqr] = IQR_col
    df_weather[col_lower] = Q1_col
    df_weather[col_upper] = Q3_col

In [50]:
df_weather.dtypes

Location              category
MinTemp                float64
MaxTemp                float64
Rainfall               float64
Evaporation            float64
Sunshine               float64
WindGustDir           category
WindGustSpeed          float64
WindDir9am            category
WindDir3pm            category
WindSpeed9am           float64
WindSpeed3pm           float64
Humidity9am            float64
Humidity3pm            float64
Pressure9am            float64
Pressure3pm            float64
Cloud9am               float64
Cloud3pm               float64
Temp9am                float64
Temp3pm                float64
RainTomorrow             int64
month                    int64
day                      int64
year                     int64
MinTemp_IQR            float64
MinTemp_lower          float64
MinTemp_upper          float64
MaxTemp_IQR            float64
MaxTemp_lower          float64
MaxTemp_upper          float64
                        ...   
WindSpeed9am_IQR       float64
WindSpee

### Identify the potential columns for outliers:

In [51]:
#df_num.drop(['month','day','year'], axis=1,inplace=True)

### create outlier column for each numerical feature in dataframe using a threshhold value and 
### np.select(condlist,choicelist )

In [52]:
threshhold_value = 2.5

choicelist = [0,0,1,1]
# use np.select(condlist,choicelist) to create outlier column, 1 if z score is greater than threshhold value
# else 0, np.select(condlist,choicelist) can process a whole series or column: very valuable
for col in num_cols:
    col_outlier = col + '_outlier'
    col_zscore = col + '_zscore'
    z_score = (df_weather[col] - df_weather[col].mean())/df_weather[col].std(ddof=0)
    condlist = [z_score == np.NaN,z_score < threshhold_value,z_score >= threshhold_value,z_score >= -1*threshhold_value]
    df_weather[col_outlier] = np.select(condlist,choicelist)
    
        
    df_weather[col_zscore] = z_score
    
    

In [53]:
df_weather.dtypes

Location                 category
MinTemp                   float64
MaxTemp                   float64
Rainfall                  float64
Evaporation               float64
Sunshine                  float64
WindGustDir              category
WindGustSpeed             float64
WindDir9am               category
WindDir3pm               category
WindSpeed9am              float64
WindSpeed3pm              float64
Humidity9am               float64
Humidity3pm               float64
Pressure9am               float64
Pressure3pm               float64
Cloud9am                  float64
Cloud3pm                  float64
Temp9am                   float64
Temp3pm                   float64
RainTomorrow                int64
month                       int64
day                         int64
year                        int64
MinTemp_IQR               float64
MinTemp_lower             float64
MinTemp_upper             float64
MaxTemp_IQR               float64
MaxTemp_lower             float64
MaxTemp_upper 

### Shortlist of numerical columns with outlier values, greater than 2.5 taken from previous notebook Milestone3

In [54]:
shortlist =  ['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm']

In [55]:
for item in shortlist:
    print(item)

MinTemp
MaxTemp
Rainfall
Evaporation
WindGustSpeed
WindSpeed9am
WindSpeed3pm
Pressure9am
Pressure3pm
Temp9am
Temp3pm


In [56]:
print(shortlist)

['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm']


In [57]:
print('The Numerical columns in weather data with outliers')
print('Z-score greater than threshhold value of ' + str(threshhold_value)+':')
print()

for col in shortlist:
    print(col)
    print('Lower bound: ' + str(df_weather[col+'_lower'][0]))
    print('Upper bound: ' +str(df_weather[col+'_upper'][0]))
    print('IQR: ' +str(df_weather[col+'_IQR'][0]))
    print()

The Numerical columns in weather data with outliers
Z-score greater than threshhold value of 2.5:

MinTemp
Lower bound: 7.7
Upper bound: 16.8
IQR: 9.100000000000001

MaxTemp
Lower bound: 18.0
Upper bound: 28.2
IQR: 10.2

Rainfall
Lower bound: 0.0
Upper bound: 1.0
IQR: 1.0

Evaporation
Lower bound: 4.0
Upper bound: 5.468231522922462
IQR: 1.468231522922462

WindGustSpeed
Lower bound: 31.0
Upper bound: 46.0
IQR: 15.0

WindSpeed9am
Lower bound: 7.0
Upper bound: 19.0
IQR: 12.0

WindSpeed3pm
Lower bound: 13.0
Upper bound: 24.0
IQR: 11.0

Pressure9am
Lower bound: 1013.5
Upper bound: 1021.8
IQR: 8.299999999999955

Pressure3pm
Lower bound: 1011.1
Upper bound: 1019.4
IQR: 8.299999999999955

Temp9am
Lower bound: 12.3
Upper bound: 21.5
IQR: 9.2

Temp3pm
Lower bound: 16.7
Upper bound: 26.2
IQR: 9.5



In [58]:
df_weather[df_weather.WindSpeed9am_outlier == 1][['WindSpeed9am','WindSpeed9am_zscore','WindSpeed9am_outlier']].head(10)

,WindSpeed9am,WindSpeed9am_zscore,WindSpeed9am_outlier
299,37.0,2.590735,1
2462,37.0,2.590735,1
3223,37.0,2.590735,1
3308,48.0,3.832126,1
3693,39.0,2.816442,1
5050,44.0,3.380711,1
5293,37.0,2.590735,1
9104,39.0,2.816442,1
9147,43.0,3.267857,1
9198,41.0,3.042150,1


### 3.Engineer outliers:

In [59]:
# drop rows from short list with z score greater than 2.5 i.e. outlier column value == 1
df_weather.shape

(145460, 104)

In [60]:
# iterate with loop over shortlist of columns with extreme values
# and drop rows where df_weather[col_outlier] == 1
for col in shortlist:
    col_outlier = col + '_outlier'
    df_weather.drop( df_weather[ df_weather[col_outlier] == 1 ].index, inplace=True)
   

In [61]:
df_weather.shape

(131584, 104)

In [62]:
# sanity check if outlier rows with value of 1 are dropped or not for one of the columns in shortlist
df_weather['WindSpeed9am_outlier'].unique().tolist()

[0]

In [63]:
df_weather.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,Pressure3pm_outlier,Pressure3pm_zscore,Cloud9am_outlier,Cloud9am_zscore,Cloud3pm_outlier,Cloud3pm_zscore,Temp9am_outlier,Temp9am_zscore,Temp3pm_outlier,Temp3pm_zscore
count,131584.000000,131584.000000,131584.000000,131584.000000,131584.000000,131584.000000,131584.000000,131584.000000,131584.000000,131584.000000,...,131584.0,1.315840e+05,131584.0,1.315840e+05,131584.0,1.315840e+05,131584.0,131584.000000,131584.0,131584.000000
mean,11.928250,23.025697,1.387437,5.176429,7.639239,38.470373,13.220123,17.931255,69.369150,51.643350,...,0.0,4.313913e-02,0.0,-9.467139e-03,0.0,-1.360976e-02,0.0,-0.035452,0.0,-0.024211
std,6.252606,6.759174,3.450684,2.228388,2.679726,11.138993,7.893649,7.831471,18.050045,19.747908,...,0.0,9.360093e-01,0.0,9.921706e-01,0.0,9.936822e-01,0.0,0.969127,0.0,0.952534
min,-8.500000,-4.100000,0.000000,0.000000,0.000000,7.000000,0.000000,0.000000,1.000000,1.000000,...,0.0,-5.260527e+00,0.0,-1.963042e+00,0.0,-2.154823e+00,0.0,-3.719940,0.0,-3.909965
25%,7.500000,18.000000,0.000000,4.000000,7.611178,31.000000,7.000000,13.000000,58.000000,38.000000,...,0.0,-5.486069e-01,0.0,-6.388874e-01,0.0,-2.436421e-01,0.0,-0.727320,0.0,-0.712900
50%,11.800000,22.600000,0.000000,5.468232,7.611178,39.000000,13.000000,17.000000,70.000000,51.539116,...,0.0,-1.705998e-14,0.0,-3.920285e-16,0.0,-8.487347e-16,0.0,-0.060571,0.0,-0.055969
75%,16.400000,27.900000,0.800000,5.468232,8.700000,44.000000,19.000000,24.000000,83.000000,65.000000,...,0.0,6.518823e-01,0.0,6.852670e-01,0.0,7.119482e-01,0.0,0.652696,0.0,0.630159
max,28.100000,40.900000,23.200000,13.200000,14.500000,72.000000,35.000000,39.000000,100.000000,100.000000,...,0.0,2.497635e+00,0.0,2.009422e+00,0.0,2.145334e+00,0.0,2.497887,0.0,2.498763


In [64]:
#df_cat.to_csv('categor_data.csv')

### 4. One-hot encode the categorical variables with sensible column names.

In [65]:
# using pandas get_dummies() method
df_dummies = pd.get_dummies(df_weather,columns=cols_cat,drop_first=True )

print(df_dummies)

        MinTemp    MaxTemp   Rainfall  Evaporation  Sunshine  WindGustSpeed  \
0          13.4  22.900000   0.600000     5.468232  7.611178       44.00000   
1           7.4  25.100000   0.000000     5.468232  7.611178       44.00000   
2          12.9  25.700000   0.000000     5.468232  7.611178       46.00000   
3           9.2  28.000000   0.000000     5.468232  7.611178       24.00000   
4          17.5  32.300000   1.000000     5.468232  7.611178       41.00000   
5          14.6  29.700000   0.200000     5.468232  7.611178       56.00000   
6          14.3  25.000000   0.000000     5.468232  7.611178       50.00000   
7           7.7  26.700000   0.000000     5.468232  7.611178       35.00000   
9          13.1  30.100000   1.400000     5.468232  7.611178       28.00000   
10         13.4  30.400000   0.000000     5.468232  7.611178       30.00000   
11         15.9  21.700000   2.200000     5.468232  7.611178       31.00000   
12         15.9  18.600000  15.600000     5.468232  

In [66]:
df = pd.concat([df_weather,df_dummies],axis=1)

In [67]:
df.shape

(131584, 297)

In [68]:
df.columns

Index(['Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       ...
       'WindDir3pm_NNW', 'WindDir3pm_NW', 'WindDir3pm_S', 'WindDir3pm_SE',
       'WindDir3pm_SSE', 'WindDir3pm_SSW', 'WindDir3pm_SW', 'WindDir3pm_W',
       'WindDir3pm_WNW', 'WindDir3pm_WSW'],
      dtype='object', length=297)

### 5. Scale the features

In [69]:
# try StandardScaler() as a first attempt at scaling numerical features
scaler = StandardScaler()

In [70]:
print(num_cols)

['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm']


In [71]:
df[num_cols].describe()

,MinTemp,MinTemp,MaxTemp,MaxTemp,Rainfall,Rainfall,Evaporation,Evaporation,Sunshine,Sunshine,...,Pressure3pm,Pressure3pm,Cloud9am,Cloud9am,Cloud3pm,Cloud3pm,Temp9am,Temp9am,Temp3pm,Temp3pm
count,131584.000000,131584.000000,131584.000000,131584.000000,131584.000000,131584.000000,131584.000000,131584.000000,131584.000000,131584.000000,...,131584.000000,131584.000000,131584.000000,131584.000000,131584.000000,131584.000000,131584.000000,131584.000000,131584.000000,131584.000000
mean,11.928250,11.928250,23.025697,23.025697,1.387437,1.387437,5.176429,5.176429,7.639239,7.639239,...,1015.543366,1015.543366,4.426013,4.426013,4.481446,4.481446,16.761997,16.761997,21.517543,21.517543
std,6.252606,6.252606,6.759174,6.759174,3.450684,3.450684,2.228388,2.228388,2.679726,2.679726,...,6.237519,6.237519,2.247858,2.247858,2.079724,2.079724,6.250090,6.250090,6.524891,6.524891
min,-8.500000,-8.500000,-4.100000,-4.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,980.200000,980.200000,0.000000,0.000000,0.000000,0.000000,-7.000000,-7.000000,-5.100000,-5.100000
25%,7.500000,7.500000,18.000000,18.000000,0.000000,0.000000,4.000000,4.000000,7.611178,7.611178,...,1011.600000,1011.600000,3.000000,3.000000,4.000000,4.000000,12.300000,12.300000,16.800000,16.800000
50%,11.800000,11.800000,22.600000,22.600000,0.000000,0.000000,5.468232,5.468232,7.611178,7.611178,...,1015.255889,1015.255889,4.447461,4.447461,4.509930,4.509930,16.600000,16.600000,21.300000,21.300000
75%,16.400000,16.400000,27.900000,27.900000,0.800000,0.800000,5.468232,5.468232,8.700000,8.700000,...,1019.600000,1019.600000,6.000000,6.000000,6.000000,6.000000,21.200000,21.200000,26.000000,26.000000
max,28.100000,28.100000,40.900000,40.900000,23.200000,23.200000,13.200000,13.200000,14.500000,14.500000,...,1031.900000,1031.900000,9.000000,9.000000,9.000000,9.000000,33.100000,33.100000,38.800000,38.800000


In [72]:
#https://stackoverflow.com/questions/38420847/apply-standardscaler-to-parts-of-a-data-set
#list for cols to scale
# num_cols

# create and fit scaler
scaler.fit(df[num_cols])

#scale selected data
df[num_cols] = scaler.transform(df[num_cols])

In [73]:
df[num_cols].head()

,MinTemp,MinTemp,MaxTemp,MaxTemp,Rainfall,Rainfall,Evaporation,Evaporation,Sunshine,Sunshine,...,Pressure3pm,Pressure3pm,Cloud9am,Cloud9am,Cloud3pm,Cloud3pm,Temp9am,Temp9am,Temp3pm,Temp3pm
0,0.235383,0.235383,-0.018597,-0.018597,-0.228198,-0.228198,0.130948,0.130948,-0.010472,-0.010472,...,-1.353647,-1.353647,1.589958,1.589958,0.013696,0.013696,0.022080,0.022080,0.043289,0.043289
1,-0.724221,-0.724221,0.306888,0.306888,-0.402077,-0.402077,0.130948,0.130948,-0.010472,-0.010472,...,-1.241422,-1.241422,0.009542,0.009542,0.013696,0.013696,0.070080,0.070080,0.426439,0.426439
2,0.155416,0.155416,0.395657,0.395657,-0.402077,-0.402077,0.130948,0.130948,-0.010472,-0.010472,...,-1.097134,-1.097134,0.009542,0.009542,-1.193165,-1.193165,0.678073,0.678073,0.257853,0.257853
3,-0.436340,-0.436340,0.735936,0.735936,-0.402077,-0.402077,0.130948,0.130948,-0.010472,-0.010472,...,-0.439818,-0.439818,0.009542,0.009542,0.013696,0.013696,0.214078,0.214078,0.763610,0.763610
4,0.891112,0.891112,1.372111,1.372111,-0.112279,-0.112279,0.130948,0.130948,-0.010472,-0.010472,...,-1.530000,-1.530000,1.145089,1.145089,1.691843,1.691843,0.166079,0.166079,1.254042,1.254042


In [74]:
df[num_cols].describe()

,MinTemp,MinTemp,MaxTemp,MaxTemp,Rainfall,Rainfall,Evaporation,Evaporation,Sunshine,Sunshine,...,Pressure3pm,Pressure3pm,Cloud9am,Cloud9am,Cloud3pm,Cloud3pm,Temp9am,Temp9am,Temp3pm,Temp3pm
count,1.315840e+05,1.315840e+05,1.315840e+05,1.315840e+05,1.315840e+05,1.315840e+05,1.315840e+05,1.315840e+05,1.315840e+05,1.315840e+05,...,1.315840e+05,1.315840e+05,1.315840e+05,1.315840e+05,1.315840e+05,1.315840e+05,1.315840e+05,1.315840e+05,1.315840e+05,1.315840e+05
mean,1.095103e-16,1.095103e-16,-4.263775e-16,-4.263775e-16,9.503855e-18,9.503855e-18,-2.478562e-16,-2.478562e-16,-3.674036e-16,-3.674036e-16,...,1.261788e-14,1.261788e-14,-5.449597e-16,-5.449597e-16,-2.576301e-16,-2.576301e-16,4.942004e-16,4.942004e-16,4.423612e-16,4.423612e-16
std,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,...,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00
min,-3.267170e+00,-3.267170e+00,-4.013183e+00,-4.013183e+00,-4.020774e-01,-4.020774e-01,-2.322956e+00,-2.322956e+00,-2.850765e+00,-2.850765e+00,...,-5.666275e+00,-5.666275e+00,-1.968998e+00,-1.968998e+00,-2.154835e+00,-2.154835e+00,-3.801879e+00,-3.801879e+00,-4.079400e+00,-4.079400e+00
25%,-7.082273e-01,-7.082273e-01,-7.435400e-01,-7.435400e-01,-4.020774e-01,-4.020774e-01,-5.279303e-01,-5.279303e-01,-1.047167e-02,-1.047167e-02,...,-6.322034e-01,-6.322034e-01,-6.343896e-01,-6.343896e-01,-2.314958e-01,-2.314958e-01,-7.139120e-01,-7.139120e-01,-7.230100e-01,-7.230100e-01
50%,-2.051147e-02,-2.051147e-02,-6.298089e-02,-6.298089e-02,-4.020774e-01,-4.020774e-01,1.309482e-01,1.309482e-01,-1.047167e-02,-1.047167e-02,...,-4.608853e-02,-4.608853e-02,9.541882e-03,9.541882e-03,1.369635e-02,1.369635e-02,-2.591928e-02,-2.591928e-02,-3.334067e-02,-3.334067e-02
75%,7.151845e-01,7.151845e-01,7.211415e-01,7.211415e-01,-1.702384e-01,-1.702384e-01,1.309482e-01,1.309482e-01,3.958485e-01,3.958485e-01,...,6.503627e-01,6.503627e-01,7.002191e-01,7.002191e-01,7.301738e-01,7.301738e-01,7.100729e-01,7.100729e-01,6.869806e-01,6.869806e-01
max,2.586411e+00,2.586411e+00,2.644461e+00,2.644461e+00,6.321254e+00,6.321254e+00,3.600630e+00,3.600630e+00,2.560257e+00,2.560257e+00,...,2.622308e+00,2.622308e+00,2.034828e+00,2.034828e+00,2.172678e+00,2.172678e+00,2.614053e+00,2.614053e+00,2.648707e+00,2.648707e+00


In [75]:
df.isna().sum()

Location          0
MinTemp           0
MaxTemp           0
Rainfall          0
Evaporation       0
Sunshine          0
WindGustDir       0
WindGustSpeed     0
WindDir9am        0
WindDir3pm        0
WindSpeed9am      0
WindSpeed3pm      0
Humidity9am       0
Humidity3pm       0
Pressure9am       0
Pressure3pm       0
Cloud9am          0
Cloud3pm          0
Temp9am           0
Temp3pm           0
RainTomorrow      0
month             0
day               0
year              0
MinTemp_IQR       0
MinTemp_lower     0
MinTemp_upper     0
MaxTemp_IQR       0
MaxTemp_lower     0
MaxTemp_upper     0
                 ..
WindDir9am_ENE    0
WindDir9am_ESE    0
WindDir9am_N      0
WindDir9am_NE     0
WindDir9am_NNE    0
WindDir9am_NNW    0
WindDir9am_NW     0
WindDir9am_S      0
WindDir9am_SE     0
WindDir9am_SSE    0
WindDir9am_SSW    0
WindDir9am_SW     0
WindDir9am_W      0
WindDir9am_WNW    0
WindDir9am_WSW    0
WindDir3pm_ENE    0
WindDir3pm_ESE    0
WindDir3pm_N      0
WindDir3pm_NE     0


In [76]:
print(df.shape)

(131584, 297)


In [78]:
df.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,Albury,0.235383,-0.018597,-0.228198,0.130948,-0.010472,W,0.496423,W,WNW,...,0,0,0,0,0,0,0,0,1,0
1,Albury,-0.724221,0.306888,-0.402077,0.130948,-0.010472,WNW,0.496423,NNW,WSW,...,0,0,0,0,0,0,0,0,0,1
2,Albury,0.155416,0.395657,-0.402077,0.130948,-0.010472,WSW,0.675973,W,WSW,...,0,0,0,0,0,0,0,0,0,1
3,Albury,-0.436340,0.735936,-0.402077,0.130948,-0.010472,NE,-1.299079,SE,E,...,0,0,0,0,0,0,0,0,0,0
4,Albury,0.891112,1.372111,-0.112279,0.130948,-0.010472,W,0.227097,ENE,NW,...,0,1,0,0,0,0,0,0,0,0


In [79]:
# create features data,X , to predict the target, RainTomorrow

X = df.drop(['RainTomorrow'],axis=1).values
y = df['RainTomorrow'].values

print(X.shape)
print()
print(y.shape)

(131584, 295)

(131584, 2)


### 1. Split the dataset into training and testing sets:

In [80]:
# split the data into training & test data with test size = 20%
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.20, random_state=4)
print(X_train.shape)
print()
print(X_test.shape)

(105267, 295)

(26317, 295)


In [81]:
print(y_train)

[[0 0]
 [0 0]
 [0 0]
 ...
 [1 1]
 [0 0]
 [1 1]]
